### 1. flask 연동하기

In [1]:
# 분석용 서버 구축을 위한 flask 설치
!pip install flask

In [2]:
# 영상처리분야에서 많이 쓰이는 라이브러리
!pip install opencv-python

In [2]:
from flask import Flask, request, Response, redirect
import io # 파이썬의 입출력을 도와주는 라이브러리
import numpy as np
import matplotlib.pyplot as plt
import cv2 # opencv
import pickle # 모델 로딩용

In [14]:
# 서버생성
app = Flask(__name__)

# 사용자의 요청에 따라 처리하는 라우터(서블릿 역활)
@app.route('/', methods=['GET','POST'])
def index():
    # get방식 처리 코드
    if request.method=='GET':
        print(request.args)
        return 'Hello {}'.format(request.args['name'])
    else:
        return 'Hello'
@app.route('/bye/<name>')    #rest API 할때 많이 활용하는 방법
def bye(name):
    return 'byebye {}'.format(name)


app.run(host='192.168.21.246', port=5001) # 서버구동   

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.21.246:5001/ (Press CTRL+C to quit)
192.168.21.246 - - [05/Oct/2022 12:21:05] "GET /bye/sj HTTP/1.1" 200 -


#### 2. 손글씨 예측 서버 구축

In [8]:
# 학습된 모델 객체 로딩
from numpy import resize


with open('./digit_model.pkl','rb') as f:
    model = pickle.load(f)
# 분석용 서버 객체 생성
app = Flask(__name__)

#route 하기전에 서버 실행 한번 해놓기
@app.route("/predict", methods=['POST'])
def predict():
    # 사진을 받아서 전처리
    if request.method == "POST":
        print(request.files) # 보낸 사진파일은 request객체안에 files에 있다
        img = request.files['img']
        input_stream = io.BytesIO() # byte단위로 읽어들이는 통로(바이너리 상태로 묶여있음)
        img.save(input_stream) # 파일로부터 데이터를 읽기
        data = np.fromstring(input_stream.getvalue(),  #0과 1로 된 데이터들 꺼내옴
                            dtype=np.uint8)  # unsigned int(음수 사용안하고 양수만 사용) 
                                             # 한픽셀당 0~255가지 이므로 1byte(8bit)만 있어도 됨
        #byte단위의 numpy 데이터를 opencv를 이용해 이미지로 변환
        real_img = cv2.imdecode(data, cv2.IMREAD_GRAYSCALE)  # 흑백은 IMREAD_GRAYSCALE(흑백으로 복원해 달라라는 의미)
                                                             # 컬러는 IMREAD_COLOR
        # # BGR -> RGB (컬러사진일 경우 필요한 단계)
        # real_img_rbg = cv2.cvtColor(real_img, cv2.COLOR_BGR2RGB) # opencv는 bgr순서로 불러옴
        # display(real_img_rbg)
        # plt.imshow(real_img_rbg)
        # plt.show()

        #모델 예측
        resize_img = cv2.resize(real_img, dsize =(28,28),  # 손글씨 크기가 28x28이므로 그거에 맞게 사이즈 변경해줌
                                interpolation=cv2.INTER_AREA) # 보간법
        # 만약에 학습데이터가 스케일링 되었으면 
        # 예측 데이터도 스테일링 작업을 해줘야함

        # 모델이 예측하기 위한 numpy타입으로 변경
        test = np.array(resize_img).reshape(1,784) # 학습데이터와 모양 동일하게 바꿔줌

        # 모델 예측
        pre = model.predict(test)

    return redirect("http://localhost:8081/HandDifitService/result.jsp?pre="+str(pre[0]))   # redirect(톰캣주소/java프로젝트파일명)
 
# 서버 실행
app.run(host='192.168.21.246', port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.21.246:5001/ (Press CTRL+C to quit)
C:\Users\pipi1\AppData\Local\Temp\ipykernel_18280\1455977616.py:19: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(input_stream.getvalue(),  #0과 1로 된 데이터들 꺼내옴
c:\Users\pipi1\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
192.168.21.246 - - [06/Oct/2022 10:38:39] "POST /predict HTTP/1.1" 302 -


ImmutableMultiDict([('img', <FileStorage: '2.jpg' ('image/jpeg')>)])
